In [5]:
!pip install torch==1.4.0 torchvision==0.5.0
import os
import sys
import torch
import zipfile
from google.colab import drive
import matplotlib.pyplot as plt

batch_size = 32
outp_size = [224,224]
load_prev_weights = True
cuda = torch.cuda.is_available()

# Paths
drive.mount('/content/drive/')

drive.mount('/content/drive/')

Train_ann_path = '/home/my_coin_database/Annotations/training.csv'
Train_img_path = '/home/my_coin_database/Images/training/'

Valid_ann_path = '/home/my_coin_database/Annotations/validation.csv'
Valid_img_path = '/home/my_coin_database/Images/validation/'

state_dict_path = '/content/drive/My Drive/Colab Notebooks/Coin_CNN/my_modelABA.pth'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
try:
  os.listdir('/home/Coin_Dataset/Images/training')
except:
  zip_ref = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Datasets/my_coin_database.zip')
  zip_ref.extractall('/home')
  zip_ref.close()

sys.path.append('/content/drive/My Drive/Colab Notebooks/Coin_CNN/')
sys.path.append('/content/drive/My Drive/Colab Notebooks/lib/')

import my_statistics
import MobileNetV2
import Dataset


In [10]:
if __name__ == '__main__':
    train_dataset = Dataset.Make(Train_img_path, Train_ann_path, output_size=outp_size)
    valid_dataset = Dataset.Make(Valid_img_path, Valid_ann_path, output_size=outp_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True, num_workers=3)
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=False,num_workers=1)

    dataloader_choice = valid_dataloader

    #model = MobileNet.Make()
    model = MobileNetV2.Make(N_classes=5,pretrained=True)
    if load_prev_weights:
        try:
            model.load_state_dict(torch.load(state_dict_path))
            print('Weights loaded successfuly!')
        except:
            print('Weights not loaded!')
            raise AssertionError
    if cuda: model = model.cuda()
    model.eval()

    Tv = [] # True Coin Value
    Mv = [] # Predicted Coin Value

    with torch.no_grad():
        for i,data in enumerate(dataloader_choice,0):
          images = data['image']
          targets= data['value']
          if cuda:
              images = images.cuda()
              targets= targets.cuda()
          outputs= [torch.argmax(item) for item in model(images)]

          Tv.extend(targets)
          Mv.extend(outputs)

          print('\r',end='')
          print(f'Progress: {(i/(len(dataloader_choice)-1))*100:.2f} %',end='')

    confusion_matrix = my_statistics.confusion_matrix(Mv,Tv,n_classes=5)
    precision,recall = my_statistics.precision_recall(confusion_matrix )

    print('\r',end='')
    print(f'Precision: {precision[0]:.2f}, {precision[1]:.2f}, {precision[2]:.2f}, {precision[3]:.2f}, {precision[4]:.2f} ')
    print(f'Recall: {recall[0]:.2f}, {recall[1]:.2f}, {recall[2]:.2f}, {recall[3]:.2f}, {recall[4]:.2f}')
    print('Confusion Matrix:')
    print(confusion_matrix.numpy())

Weights loaded successfuly!
Precision: 0.98, 0.95, 0.97, 0.98, 1.00 
Recall: 0.98, 0.97, 0.98, 0.97, 0.98
Confusion Matrix:
[[49.  0.  1.  0.  0.]
 [ 1. 58.  0.  1.  0.]
 [ 0.  1. 65.  0.  0.]
 [ 0.  2.  0. 63.  0.]
 [ 0.  0.  1.  0. 58.]]


In [11]:
input_tensor = torch.rand(1,3,224,224)

script_model = torch.jit.trace(model.cpu(),input_tensor)
script_model.save(state_dict_path+'.pt')